# Context of Dataset
Company - UK-based and registered non-store online retail

Products for selling - Mainly all-occasion gifts

Customers - Most are wholesalers (local or international)

Transactions Period - **1st Dec 2010 - 9th Dec 2011 (One year)**

* #  *Loading the dataset & preparing dataframe for EDA*

IMPORTANT: Run the below cell & go to "Run-> Restart & clear cell outputs", before proceeding further

In [ ]:
# !pip list | grep numpy
# !pip list | grep pandas
!pip install pandas==1.1.3
!pip install numpy==1.18.5

from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:


import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
# current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

import missingno as msno # missing data visualization module for Python
import pandas_profiling

import gc
import datetime

%matplotlib inline
color = sns.color_palette()
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format
# specify encoding to deal with different formats
df = pd.read_csv('../input/ecommerce-data/data.csv', encoding = 'ISO-8859-1')
# change the column names
df.rename(index=str, columns={'InvoiceNo': 'invoice_num',
                              'StockCode' : 'stock_code',
                              'Description' : 'description',
                              'Quantity' : 'quantity',
                              'InvoiceDate' : 'invoice_date',
                              'UnitPrice' : 'unit_price',
                              'CustomerID' : 'cust_id',
                              'Country' : 'country'}, inplace=True)
# Checking the missing values in all the columns

print("Missing values in all the columns:")
print(df.isna().sum())

# Replace missing values with a default value
df.description.fillna('No description', inplace=True) #inplace will replace the column in same dataframe and wont return a new dataframe
print("\nMissing values in all the columns after replacement:")
print(df.isna().sum())

# Replace missing values with a mean value
df.cust_id.fillna(np.mean(df.cust_id), inplace=True) 
print("\nMissing values in all the columns after replacement:")
print(df.isna().sum())
# change the invoice_date format - String to Timestamp format
df['invoice_date'] = pd.to_datetime(df.invoice_date, format='%m/%d/%Y %H:%M')
# change description - UPPER case to LOWER case
df['description'] = df.description.str.lower()

# df_new without missing values
df_new = df.dropna()

# check missing values for each column 
df_new.isnull().sum().sort_values(ascending=False)

# change columns tyoe - String to Int type 
df_new['cust_id'] = df_new['cust_id'].astype('int64')

df_new = df_new[df_new.quantity > 0]
df_new = df_new[df_new.unit_price >= 0]

df_new['amount_spent'] = df_new['quantity'] * df_new['unit_price']

# rearrange all the columns for easy reference
df_new = df_new[['invoice_num','invoice_date','stock_code','description','quantity','unit_price','amount_spent','cust_id','country']]
df_new.insert(loc=2, column='year_month', value=df_new['invoice_date'].map(lambda x: 100*x.year + x.month))
df_new.insert(loc=3, column='month', value=df_new.invoice_date.dt.month)
# +1 to make Monday=1.....until Sunday=7
df_new.insert(loc=4, column='day', value=(df_new.invoice_date.dt.dayofweek)+1)
df_new.insert(loc=5, column='hour', value=df_new.invoice_date.dt.hour)



# Exploratory Data Analysis (EDA)

## How many orders placed by the customers?

In [ ]:
orders = df_new.groupby(by=['cust_id','country'], as_index=False)['invoice_num'].count()

orders = orders.sort_values(by='invoice_num', ascending=False).head()
orders.cust_id = orders.cust_id.astype('str')
ax = orders.plot(kind='bar',x='cust_id', y="invoice_num",color=color[0],figsize=(15,6))
ax.set_xticklabels(orders.cust_id)
plt.xlabel('Customers ID')
plt.ylabel('Number of Orders')
plt.title('Number of Orders for different Customers')


plt.show()

### Check TOP 5 most number of orders

In [ ]:
print('The TOP 5 customers with most number of orders...')
orders.sort_values(by='invoice_num', ascending=False).head()

## How much money spent by the customers?

In [ ]:
money_spent = df_new.groupby(by=['cust_id','country'], as_index=False)['amount_spent'].sum()

money_spent = money_spent.sort_values(by='amount_spent', ascending=False).head()
money_spent.cust_id = money_spent.cust_id.astype('str')
ax = money_spent.plot(kind='bar',x='cust_id', y="amount_spent",color=color[0],figsize=(15,6))
ax.set_xticklabels(money_spent.cust_id)
plt.xlabel('Customers ID')
plt.ylabel('Money spent (Dollar)')
plt.title('Money Spent by different Customers')

ax.ticklabel_format(useOffset=False, style='plain', axis='y') # Used to prevent exponential notation in y-axis ticks

plt.show()

### Check top 5 highest money spenders

In [ ]:
print('The top 5 customers with highest money spent...')
print(money_spent.sort_values(by='amount_spent', ascending=False).head())

# Discover patterns for Number of Orders
Credit: https://www.kaggle.com/hendraherviawan/customer-purchasing-patterns

## How many orders (per month)?

In [ ]:
ax = df_new.groupby('invoice_num')['year_month'].unique().value_counts().sort_index().plot(kind='bar',color=color[0],figsize=(15,6))
ax.set_xlabel('Month',fontsize=15)
ax.set_ylabel('Number of Orders',fontsize=15)
ax.set_title('Number of orders for different Months (1st Dec 2010 - 9th Dec 2011)',fontsize=15)
ax.set_xticklabels(('Dec_10','Jan_11','Feb_11','Mar_11','Apr_11','May_11','Jun_11','July_11','Aug_11','Sep_11','Oct_11','Nov_11','Dec_11'), rotation='horizontal', fontsize=13)
plt.show()

## How many orders (per day)?

In [ ]:
ax = df_new.groupby('invoice_num')['day'].unique().value_counts().sort_index().plot(kind='bar',color=color[0],figsize=(15,6))
ax.set_xlabel('Day of the week',fontsize=15)
ax.set_ylabel('Number of Orders',fontsize=15)
ax.set_title('Number of orders for different Days',fontsize=15)
ax.set_xticklabels(('Mon','Tue','Wed','Thu','Fri','Sun'), rotation='horizontal', fontsize=15)
plt.show()

 ## How many orders (per hour)?


In [ ]:
ax = df_new.groupby('invoice_num')['hour'].unique().value_counts().iloc[:-1].sort_index().plot(kind='bar',color=color[0],figsize=(15,6))
ax.set_xlabel('Hour',fontsize=15)
ax.set_ylabel('Number of Orders',fontsize=15)
ax.set_title('Number of orders for different Hours',fontsize=15)
ax.set_xticklabels(range(6,21), rotation='horizontal', fontsize=15)
plt.show()

# Discover patterns for Unit Price

In [ ]:
df_new.unit_price.describe()

**We see that there are unit price = 0 (FREE items)**

<span class="mark">There are some free items given to customers from time to time</span>.

In [ ]:
# check the distribution of unit price
plt.subplots(figsize=(12,6))
sns.boxplot(df_new.unit_price)
plt.show()

In [ ]:
df_free = df_new[df_new.unit_price == 0]

In [ ]:
df_free.head()

In [ ]:
df_free.year_month.value_counts().sort_index()

In [ ]:

ax = df_free.year_month.value_counts().sort_index().plot(kind='bar',figsize=(12,6), color=color[0])
ax.set_xlabel('Month',fontsize=15)
ax.set_ylabel('Frequency',fontsize=15)
ax.set_title('Frequency for different Months (Dec 2010 - Dec 2011)',fontsize=15)
ax.set_xticklabels(('Dec_10','Jan_11','Feb_11','Mar_11','Apr_11','May_11','June_11','July_11','Aug_11','Sep_11','Oct_11','Nov_11','Dec_11'), rotation='horizontal', fontsize=13)
plt.show()

**Not clear why there are FREE items given to certain customers**
- On average, the company <span class="mark">gave out 2-4 times FREE items</span> to customers <span class="mark">each month (Except in June 2011)</span>

# Discover patterns for each Country

In [ ]:
df_new.head()

## How many orders for each country?

In [ ]:
group_country_orders = df_new.groupby('country')['invoice_num'].count().sort_values()
# del group_country_orders['United Kingdom']

# plot number of unique customers in each country (with UK)
plt.subplots(figsize=(15,8))
group_country_orders.plot(kind='barh', fontsize=12, color=color[0])
plt.xlabel('Number of Orders', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Number of Orders for different Countries', fontsize=12)
plt.show()

In [ ]:
group_country_orders = df_new.groupby('country')['invoice_num'].count().sort_values()
del group_country_orders['United Kingdom']

# plot number of unique customers in each country (without UK)
plt.subplots(figsize=(15,8))
group_country_orders.plot(kind='barh', fontsize=12, color=color[0])
plt.xlabel('Number of Orders', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Number of Orders for different Countries', fontsize=12)
plt.show()

## How much money spent by each country?

In [ ]:
group_country_amount_spent = df_new.groupby('country')['amount_spent'].sum().sort_values()
# del group_country_orders['United Kingdom']

# plot total money spent by each country (with UK)
plt.subplots(figsize=(15,8))
group_country_amount_spent.plot(kind='barh', fontsize=12, color=color[0])
plt.xlabel('Money Spent (Dollar)', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Money Spent by different Countries', fontsize=12)
plt.show()

In [ ]:
group_country_amount_spent = df_new.groupby('country')['amount_spent'].sum().sort_values()
del group_country_amount_spent['United Kingdom']

# plot total money spent by each country (without UK)
plt.subplots(figsize=(15,8))
group_country_amount_spent.plot(kind='barh', fontsize=12, color=color[0])
plt.xlabel('Money Spent (Dollar)', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Money Spent by different Countries', fontsize=12)
plt.show()

# Results obtained from Exploratory Data Analysis (EDA)

1. The **customer with the highest number of orders comes from the United Kingdom (UK)** 
2. The **customer with the highest money spent on purchases comes from Netherlands** 
3. The company receives the highest number of orders from customers in the UK (since it is a UK-based company). Therefore, the TOP 5 countries (including UK) that place the highest number of orders are as below: 
    - United Kingdom 
	- Germany 
	- France
	- Ireland (EIRE)
	- Spain
4. As the company receives the highest number of orders from customers in the UK (since it is a UK-based company), customers in the UK spend the most on their purchases. Therefore, the TOP 5 countries (including UK) that spend the most money on purchases are as below: 
    - United Kingdom 
	- Netherlands
	- Ireland (EIRE)
	- Germany
	- France
5. **November 2011 has the highest sales** 
	- The month with the lowest sales is undetermined as the dataset consists of transactions until 9th December 2011 in December 
6. There are **no transactions on Saturday** between 1st Dec 2010 - 9th Dec 2011 
7. The number of orders received by the company tends to increases from Monday to Thursday and decrese afterward
8. The company receives the **highest number of orders at 12:00pm** 
	- Possibly most customers made purchases during **lunch hour between 12:00pm - 2:00pm**
9. The company tends to **give out FREE items for purchases occasionally each month (Except June 2011)** 
	- However, it is not clear what factors contribute to giving out the FREE items to the particular customers 